In [1]:
import pandas as pd
import stateplane

In [2]:
# Read raw data locally
df = pd.read_csv("citation_data_raw.csv", low_memory=False)

In [3]:
display(df)

,ticket_number,issue_date,issue_time,rp_state_plate,plate_expiry_date,make,body_style,color,location,route,...,fine_amount,latitude,longitude,agency_desc,color_desc,body_style_desc,violation_code,violation_description,meter_id,marked_time
0,1127615064,2020-01-01T00:00:00.000,1220.0,CA,202010.0,VOLV,TC,BL,ALAMEDA ST/COLON ST,M74,...,93.0,9.999900e+04,9.999900e+04,VALLEY,BLUE,NaN,NaN,NaN,NaN,NaN
1,1127615031,2020-01-01T00:00:00.000,1155.0,CA,NaN,GALT,TL,BL,1715 E DENNI ST,M74,...,250.0,6.488337e+06,1.744508e+06,VALLEY,BLUE,NaN,NaN,NaN,NaN,NaN
2,1127620675,2020-01-01T00:00:00.000,1015.0,CA,202003.0,HOND,PA,WH,BERTH 79 PARKING LOT,00005,...,68.0,9.999900e+04,9.999900e+04,VALLEY,NaN,PASSENGER CAR,NaN,NaN,NaN,NaN
3,1127620130,2020-01-01T00:00:00.000,2352.0,CA,201906.0,FORD,PA,GY,MARINE AVE & HARRY B,2L27,...,50.0,9.999900e+04,9.999900e+04,VALLEY,GREY,PASSENGER CAR,NaN,NaN,NaN,NaN
4,1127615075,2020-01-01T00:00:00.000,1230.0,CA,NaN,WABA,TL,WH,SANDISON ST/GAMBLE,M74,...,250.0,9.999900e+04,9.999900e+04,VALLEY,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697626,4562911916,2023-03-27T00:00:00.000,727.0,CA,202310.0,NISS,PU,WT,3101 6TH ST W,NaN,...,0.0,6.474310e+06,1.845688e+06,54 - DOT - HOLLYWOOD,WHITE,PICK-UP TRUCK,NaN,NaN,NaN,800.0
5697627,4564280361,2023-03-27T00:00:00.000,943.0,CA,0.0,NaN,PA,NaN,5700 HOOVER ST S,NaN,...,0.0,6.474637e+06,1.819061e+06,55 - DOT - SOUTHERN,NaN,PASSENGER CAR,NaN,NaN,NaN,0.0
5697628,4564689511,2023-03-27T00:00:00.000,1352.0,CA,202307.0,TOYT,PA,WT,4575 WESTLAWN AVE,NaN,...,0.0,NaN,NaN,51 - DOT - WESTERN,WHITE,PASSENGER CAR,NaN,NaN,NaN,0.0
5697629,4564713650,2023-03-27T00:00:00.000,2231.0,CA,11.0,FORD,PA,WT,12815 HERRICK AVE,NaN,...,0.0,NaN,NaN,53 - DOT - VALLEY,WHITE,PASSENGER CAR,NaN,NaN,NaN,0.0


# Preprocessing

In [4]:
# Check the number of missing values
df.isna().sum()

ticket_number                  0
issue_date                     0
issue_time                   684
rp_state_plate              2762
plate_expiry_date         370595
make                       34776
body_style                 10857
color                      51358
location                    5446
route                    2156768
agency                         1
fine_amount                55967
latitude                   76704
longitude                  76704
agency_desc                    1
color_desc                182483
body_style_desc            44243
violation_code            303133
violation_description     461134
meter_id                 4714810
marked_time              3043593
dtype: int64

In [5]:
# Remove entires with missing issue time
df = df[df['issue_time'].notna()]
# Remove entries with missing latitude/longitude
df = df[df['latitude'].notna()]
# Remove entries with missing violation code
df = df[df['violation_code'].notna()]
# Remove latitude/longitude entries with 99999 value (invalid)
df = df[df['latitude'] != 99999]
df = df[df['longitude'] != -99999]

In [6]:
# Inspect data types
df.dtypes

ticket_number             object
issue_date                object
issue_time               float64
rp_state_plate            object
plate_expiry_date        float64
make                      object
body_style                object
color                     object
location                  object
route                     object
agency                   float64
fine_amount              float64
latitude                 float64
longitude                float64
agency_desc               object
color_desc                object
body_style_desc           object
violation_code            object
violation_description     object
meter_id                  object
marked_time              float64
dtype: object

In [7]:
# Convert issue date columns to datetime
df['issue_date'] = df['issue_date'].apply(pd.to_datetime)
# Format issue time
df = df.astype({'issue_time': int})

Since the latitude/longitude (XY) is in US Feet coordinates according to the California State Plane Coordinate System - Zone 5 (https://www.conservation.ca.gov/cgs/rgm/state-plane-coordinate-system), we want to convert it to lat/long system by using `stateplane` with project description [here](https://pypi.org/project/stateplane/).

In [8]:
%%time
lat = list()
long = list()
for index, row in df.iterrows():
    work_coor = stateplane.to_latlon(row['latitude'], row['longitude'], epsg='2229')
    lat.append(work_coor[0])
    long.append(work_coor[1])

Wall time: 10min


In [9]:
# Add lat/lon information to dataframe and remove the old coordinates
df['lat'] = lat
df['lon'] = long
df = df.drop(columns = ['latitude', 'longitude'])

In [10]:
display(df)

,ticket_number,issue_date,issue_time,rp_state_plate,plate_expiry_date,make,body_style,color,location,route,...,fine_amount,agency_desc,color_desc,body_style_desc,violation_code,violation_description,meter_id,marked_time,lat,lon
15,4508182571,2020-01-01,228,CA,NaN,VOLK,PA,BK,1729 CLINTON ST,00600,...,68.0,56 - DOT - CENTRAL,BLACK,PASSENGER CAR,80.61,STANDNG IN ALLEY,NaN,NaN,34.072180,-118.262252
16,4508195565,2020-01-01,736,CA,NaN,NISS,PA,GY,700 76TH ST W,00500,...,68.0,55 - DOT - SOUTHERN,GREY,PASSENGER CAR,22514,FIRE HYDRANT,NaN,NaN,33.970746,-118.285640
23,1109775155,2020-01-01,100,CA,202002.0,DODG,PA,BL,328 LORENA,NaN,...,68.0,WESTERN,BLUE,PASSENGER CAR,22500F,NaN,NaN,NaN,34.034616,-118.196633
24,1110713262,2020-01-01,1830,CA,201909.0,NISS,PA,BK,2800 E OBSERVATORY,NaN,...,68.0,HOLLYWOOD,BLACK,PASSENGER CAR,22514,NaN,NaN,NaN,34.123408,-118.302409
25,1110713273,2020-01-01,1900,CA,NaN,FORD,PA,GY,2800 E OBSERVATORY,NaN,...,68.0,HOLLYWOOD,GREY,PASSENGER CAR,22514,NaN,NaN,NaN,34.123408,-118.302409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697585,4563783313,2023-03-27,2131,CA,202302.0,VOLK,PA,WT,2118 6TH AVE,0205B,...,68.0,51 - DOT - WESTERN,WHITE,PASSENGER CAR,22500E,BLOCKING DRIVEWAY,NaN,0.0,34.037828,-118.323630
5697586,4564468904,2023-03-27,2051,CA,0.0,MERZ,PA,WT,3901 BLUE CANYON DR,350R1,...,68.0,53 - DOT - VALLEY,WHITE,PASSENGER CAR,80.58L,PREFERENTIAL PARKING,NaN,0.0,34.138853,-118.385230
5697588,4563965055,2023-03-27,2227,CA,202401.0,TOYT,PA,BK,15202 VICTORY BLVD,NaN,...,68.0,53 - DOT - VALLEY,BLACK,PASSENGER CAR,80.61,STANDNG IN ALLEY,NaN,0.0,34.186651,-118.464072
5697589,4564298384,2023-03-27,2123,CA,202004.0,NISS,VN,WT,600 PARK ROW DR,NaN,...,73.0,56 - DOT - CENTRAL,WHITE,VAN,80.69B,NO PARKING,NaN,0.0,34.076598,-118.231198


In [11]:
# Reset index
df.reset_index(drop=True, inplace=True)

In [13]:
# Save dataset
df.to_csv("citation_data_complete.csv", index=False, encoding='utf-8')